# Traffic Violation RAG System
In this exam, you will implement a Retrieval-Augmented Generation (RAG) system that uses a language model and a vector database to answer questions about traffic violations. The goal is to generate answers with relevant data based on a dataset of traffic violations and fines.

Here are helpful resources:
* [LangChain](https://www.langchain.com/)
* [groq cloud documentation](https://console.groq.com/docs/models)
* [LangChain HuggingFace](https://python.langchain.com/docs/integrations/text_embedding/sentence_transformers/)
* [Chroma Vector Store](https://python.langchain.com/docs/integrations/vectorstores/chroma/)
* [Chroma Website](https://docs.trychroma.com/getting-started)
* [ChatGroq LangChain](https://python.langchain.com/docs/integrations/chat/groq/)
* [LLM Chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain)

Dataset [source](https://www.moi.gov.sa/wps/portal/Home/sectors/publicsecurity/traffic/contents/!ut/p/z0/04_Sj9CPykssy0xPLMnMz0vMAfIjo8ziDTxNTDwMTYy83V0CTQ0cA71d_T1djI0MXA30gxOL9L30o_ArApqSmVVYGOWoH5Wcn1eSWlGiH1FSlJiWlpmsagBlKCQWqRrkJmbmqRqUZebngB2gUJAKdERJZmqxfkG2ezgAhzhSyw!!/)

Some installs if needed:
```python
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain
```

In [1]:
!kaggle datasets download -d khaledzsa/dataset
!unzip dataset.zip

Dataset URL: https://www.kaggle.com/datasets/khaledzsa/dataset
License(s): unknown
  0% 0.00/3.73k [00:00<?, ?B/s]
100% 3.73k/3.73k [00:00<00:00, 6.70MB/s]
Archive:  dataset.zip
  inflating: Dataset.csv             


## Step 1: Install Required Libraries

To begin, install the necessary libraries for this project. The libraries include `LangChain` for building language model chains, and `Chroma` for managing a vector database.

In [2]:
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain

  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement LLMChain (from versions: none)
ERROR: No matching distribution found for LLMChain


# Step 2: Load the Traffic Violations Dataset

You are provided with a dataset of traffic violations. Load the CSV file into a pandas DataFrame and preview the first few rows of the dataset using `.head()`. You can also try and see the dataset's characteristics.

In [3]:
import pandas as pd

df=pd.read_csv("/content/Dataset.csv")
df.head()

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   المخالفة  104 non-null    object
 1   الغرامة   104 non-null    object
dtypes: object(2)
memory usage: 1.8+ KB


In [28]:
df['المخالفة'].str.split().str.len().describe()

,المخالفة
count,104.000000
mean,10.413462
std,5.228025
min,3.000000
25%,6.750000
50%,9.000000
75%,13.250000
max,25.000000


In [29]:
df['الغرامة'].str.split().str.len().describe()

,الغرامة
count,104.0
mean,6.0
std,0.0
min,6.0
25%,6.0
50%,6.0
75%,6.0
max,6.0


In [5]:
df.duplicated().sum()

0

In [6]:
len(df)

104

## Step 3: Create Markdown Content from the Dataset

For each traffic violation in the dataset, you will generate markdown text that describes the violation and the associated fine. Create a loop to iterate through the dataset and store the generated markdown in a list. Each fine should look like this:

**المخالفة** - الغرامة

In [7]:
import os
Directory = 'RAG_Exam/Markdown_Files'
os.makedirs(Directory, exist_ok=True)

In [8]:
for i in range(0, 104):

    Violation = df['المخالفة'].iloc[i]
    Fine = df['الغرامة'].iloc[i]

    markdown_file = f"# {Violation}--"
    markdown_file += f"{Fine}"

    with open(f'{Directory}/{i}.md', 'w', encoding='utf-8') as file:
        file.write(markdown_file)

## Step 4: Chunk the Markdown Data

Using LangChain's `RecursiveCharacterTextSplitter`, split the markdown texts into smaller chunks that will be stored in the vector database.

In [9]:
!pip install langchain

  Using cached langchain-0.3.0-py3-none-any.whl.metadata (7.1 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 894.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.0/290.0 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [10]:
import markdown
MarkdownTexts = []
for filename in os.listdir(Directory):
  if filename.endswith(".md"):
    with open(os.path.join(Directory, filename), 'r', encoding='utf-8') as file:
      markdown_file = file.read()
      html_file = markdown.markdown(markdown_file)
      MarkdownTexts.append(html_file)

In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=30,
)
Violations =text_splitter.create_documents(MarkdownTexts)


In [12]:
Violations[0]

Document(metadata={}, page_content='<h1>ترك الأطفال دون سن العاشرة في المركبة دون مرافق راشد.--الغرامة المالية 300 - 500 ريال</h1>')

In [13]:
Violations[0].page_content

'<h1>ترك الأطفال دون سن العاشرة في المركبة دون مرافق راشد.--الغرامة المالية 300 - 500 ريال</h1>'

In [14]:
len(Violations[0].page_content)

94

## Step 5: Generate Embeddings for the Documents

Generate embeddings for the chunks of text using HuggingFace's pre-trained Arabic language model. These embeddings will be stored in a `Chroma` vector store.

In [15]:
!pip install langchain_huggingface

  Using cached langchain_huggingface-0.1.0-py3-none-any.whl.metadata (1.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 6.2 MB/s eta 0:00:00


In [16]:
!pip install langchain-community

  Using cached langchain_community-0.3.0-py3-none-any.whl.metadata (2.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.1 MB/s eta 0:00:00


In [33]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="Omartificial-Intelligence-Space/GATE-AraBert-v0")

In [19]:
from langchain_chroma import Chroma
db = Chroma.from_documents(Violations, embeddings, persist_directory="./chroma_RAG_Exam")

# Step 6: Define the RAG Prompt Template

Define a custom prompt template in Arabic to retrieve traffic violation-related
answers based on the context. Ensure the template encourages the model to give **advice** in **Arabic**, staying within the context provided.

In [31]:
from langchain.prompts import PromptTemplate
PROMPT_TEMPLATE = """
# التعليمات
•⁠  ⁠أنت خبير في تقديم الاستشارات المتعلقة بالمخالفات المرورية.
•⁠  ⁠سيتم تزويدك دائمًا بسياق يخص المخالفات المرورية.
•⁠  ⁠يجب عليك دائمًا الالتزام بالسياق وعدم اختلاق اجابه
•⁠  ⁠إذا كان السياق  غير ذي صلة، يجب أن تخبر المستخدم بأنك لا تعرف الاجابه
•⁠  ⁠تأكد من تقديم إجابتك باللغة العربية، ويجب أن تكون الإجابة واضحة ومهنية.
•⁠  ⁠لاتجيب المستخدم ابدا اذا سألك عن شي خارج النطاق
اعتمد فقط على السياق التالي للإجابة على السؤال:
السياق: {context}
السؤال: {question}
إجابتك:


"""

prompt_template = PromptTemplate(
    template=PROMPT_TEMPLATE, input_variables=["context", "question"]
)

## Step 7: Initialize the Language Model

Initialize the language model using the Groq API. Set up the model with a specific configuration, including the API key, temperature setting, and model name.

In [21]:
!pip install langchain-groq

  Using cached langchain_groq-0.2.0-py3-none-any.whl.metadata (2.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 2.6 MB/s eta 0:00:00


In [22]:
from langchain_groq import ChatGroq
groq_api_key = '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'
llm = ChatGroq(temperature=0.5,
               groq_api_key=groq_api_key,
               model_name="llama3-8b-8192")


## Step 8: Create the LLM Chain

Now, you will create an LLM Chain that combines the language model and the prompt template you defined. This chain will be used to generate responses based on the retrieved context.

In [23]:
from langchain.chains import LLMChain
MODEL = LLMChain(llm=llm,
                 prompt=prompt_template,
                 verbose=True)

<ipython-input-23-24540ef20189>:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  MODEL = LLMChain(llm=llm,


## Step 9: Implement the Query Function

Create a function `query_rag` that will take a user query as input, retrieve relevant context from the vector store, and use the language model to generate a response based on that context.

In [24]:
def query_rag(query: str):
    SimilaritySearchResults = db.similarity_search_with_score(query, k=4)
    context_text = "\n\n".join([doc.page_content for doc, _score in SimilaritySearchResults])

    rag_response = MODEL.invoke({"context": context_text, "question": query})

    return rag_response


## Step 10: Inference - Running Queries in the RAG System

In this final step, you will implement an inference pipeline to handle real-time queries. You will allow the system to retrieve the most relevant violations and fines based on a user's input and generate a response.

1. Inference Workflow:

  * The user inputs a query (e.g., "ماهي الغرامة على القيادة بدون رخصة؟").
  * The system searches for the most relevant context from the traffic violation vector store.
  * It generates an answer and advice based on the context.

2. Goal:
  * Run the inference to answer questions based on the traffic violation dataset.

In [32]:
Res = query_rag("ماهي الغرامة على القيادة بدون رخصة؟")



Prompt after formatting:

# التعليمات
•⁠  ⁠أنت خبير في تقديم الاستشارات المتعلقة بالمخالفات المرورية.
•⁠  ⁠سيتم تزويدك دائمًا بسياق يخص المخالفات المرورية.
•⁠  ⁠يجب عليك دائمًا الالتزام بالسياق وعدم اختلاق اجابه
•⁠  ⁠إذا كان السياق  غير ذي صلة، يجب أن تخبر المستخدم بأنك لا تعرف الاجابه
•⁠  ⁠تأكد من تقديم إجابتك باللغة العربية، ويجب أن تكون الإجابة واضحة ومهنية. 
•⁠  ⁠لاتجيب المستخدم ابدا اذا سألك عن شي خارج النطاق
اعتمد فقط على السياق التالي للإجابة على السؤال:
السياق: <h1>قيادة مركبة دون توافر التجهيزات اللازمة، مثل المكابح والأنوار وما في حكمهما.--الغرامة المالية

<h1>قيادة المركبة برخصة قيادة لا تتناسب مع حجم المركبة ونوع استخدامها.--الغرامة المالية 1000 - 2000

<h1>إجراء سباق للمركبات على الطرق العامة، أو السير في مواكب دون الحصول على تصريح.--الغرامة المالية

<h1>وضع كتابة أو رسم أو ملصق أو أي بيان آخر على جسم المركبة دون موافقة الجهات المختصة.--الغرامة
السؤال: ماهي الغرامة على القيادة بدون رخصة؟
إجابتك:


> Finished chain.


In [26]:
print(f'Context:\n{Res["context"]}\n\nQuestion:\n{Res["question"]}\n\nText: \n{Res["text"]}')


Context:
<h1>قيادة مركبة دون توافر التجهيزات اللازمة، مثل المكابح والأنوار وما في حكمهما.--الغرامة المالية

<h1>قيادة المركبة برخصة قيادة لا تتناسب مع حجم المركبة ونوع استخدامها.--الغرامة المالية 1000 - 2000

<h1>إجراء سباق للمركبات على الطرق العامة، أو السير في مواكب دون الحصول على تصريح.--الغرامة المالية

<h1>وضع كتابة أو رسم أو ملصق أو أي بيان آخر على جسم المركبة دون موافقة الجهات المختصة.--الغرامة

Question:
ماهي الغرامة على القيادة بدون رخصة؟

Text: 
الغرامة المالية على القيادة بدون رخصة تتراوح بين 1000 إلى 2000 ريال.


In [30]:
Res2 = query_rag("ماهي اكادمية طويق")
print(f'Context:\n{Res2["context"]}\n\nQuestion:\n{Res2["question"]}\n\nText: \n{Res2["text"]}')

Prompt after formatting:

# التعليمات
•⁠  ⁠أنت خبير في تقديم الاستشارات المتعلقة بالمخالفات المرورية.
•⁠  ⁠سيتم تزويدك دائمًا بسياق يخص المخالفات المرورية.
•⁠  ⁠يجب عليك دائمًا الالتزام بالسياق وعدم اختلاق اجابه
•⁠  ⁠إذا كان السياق  غير ذي صلة، يجب أن تخبر المستخدم بأنك لا تعرف الاجابه
•⁠  ⁠تأكد من تقديم إجابتك باللغة العربية، ويجب أن تكون الإجابة واضحة ومهنية. 
•⁠  ⁠لاتجيب المستخدم ابدا اذا سألك عن شي خارج النطاق
اعتمد فقط على السياق التالي للإجابة على السؤال:
السياق: <h1>المجازفة بعبور الأودية والشعاب أثناء جريانها.--الغرامة المالية 5000 - 10000 ريال</h1>

<h1>القيام بأعمال الطرق قبل التنسيق مع الإدارة المختصة.--الغرامة المالية 5000 - 10000 ريال</h1>

<h1>عدم التزام الشاحنات والمعدات الثقيلة بالسير في المسار الآمن في الطريق المتعدد المسارات.--الغرامة

تجهيزاتها الأساسية دون اتخاذ الإجراءات النظامية.--الغرامة المالية 1000 - 2000 ريال</h1>
السؤال: ماهي اكادمية طويق
إجابتك:


> Finished chain.
Context:
<h1>المجازفة بعبور الأودية والشعاب أثناء جريانها.--الغرامة المالية 5000 - 10000 ريال<